<a href="https://colab.research.google.com/github/Saitou0813/DeepLearning2022Late/blob/main/%E5%8F%8E%E9%9B%86%E3%81%97%E3%81%9F%E7%94%BB%E5%83%8F%E3%81%A7%E5%AD%A6%E7%BF%92%E3%81%A8%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 収集した画像をColabにUPして学習させ、それをもとに分類させる

In [ ]:
import os
os.makedirs('sample', exist_ok=True)
os.makedirs('output/dog/sample', exist_ok=True)
os.makedirs('output/cat/sample', exist_ok=True)
os.makedirs('img/deep_learning/dog', exist_ok=True)
os.makedirs('img/deep_learning/cat', exist_ok=True)

## 必要なライブラリをimport

In [ ]:
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

## 分類（クラス）の名前をフォルダ名（dog,cat）から引用する

In [ ]:
path = "img/deep_learning"
folders = os.listdir(path)
classes = [f for f in folders if os.path.isdir(os.path.join(path, f))]
print(classes)
n_classes = len(classes)
print(n_classes)

['dog', 'cat']
2


## 画像を読み込みリサイズ、整形する

In [ ]:
# 画像とラベルのための配列
X = [] # 画像(学習用データの配列)
Y = [] # ラベル(正解用の配列)

# 画像を読み込みつつリサイズする
for label, class_name in enumerate(classes):
  files = glob.glob( path + "/" + class_name + "/*jpg")
  for file in files:
    img = cv2.imread(file)
    img = cv2.resize( img, dsize=(224,224))
    X.append(img)
    Y.append(label)
    # 内部ループ終わり
# 外部ループ終わり

## 0～255のビットデータ値を、学習のために0～1に変換（正規化）学習精度を上げる

In [ ]:
X = np.array( X ) # 配列XをNumpy配列に変換
X = X.astype('float32')# 少数型に変換
X /= 255.0
# ラベルもnumpyの配列に変換
Y = np.array( Y )
Y = np_utils.to_categorical( Y, n_classes)

## 整形したデータを学習用とテスト検証用に振り分ける

In [ ]:
# 学習データ8割、テスト検証用が2割に分ける
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2)
# 学習用データ8割(画像問題データ)
print(X_train.shape)
# テストデータ2割(画像問題データ)
print( X_test.shape )
# 学習用データ8割(正解ラベル)
print( Y_train.shape )
# テスト用データ2割(正解ラベル)
print( Y_test.shape )

(120, 224, 224, 3)
(31, 224, 224, 3)
(120, 2)
(31, 2)


## 準備したデータを学習させていく

###学習に必要なライブラリをimport

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential,Model
from keras.layers import Input,Activation,Dense,Flatten,Dropout
from keras.optimizers import Adam

## モデルのVGG16をアレンジしていく

In [ ]:
# VGG16 入力層を用意する（定義する）
input_tensor = Input(shape=(224,224,3))
# VGG16のインスタンスを作る（最後のsoftmaxの1000層を無効にして省く）
base_model = VGG16( weights='imagenet', input_tensor=input_tensor, include_top=False)

58889256/58889256 [==============================] - 3s 0us/step


### 1000分類のsoftmaxを外した代わりに、2つに分類するsoftmax層を追加する

In [ ]:
# 追加する層の作成
top_model = Sequential( ) # ラッピングする層
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes,activation='softmax'))

## 最後のsoftmaxを外したモデル（basemodel）と追加用の2つに分類するsoftmax層のモデル（top_model）をつなげる

In [ ]:
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

## 時間短縮のために、15層までを外す

In [ ]:
# ループしながら15層までを無効にする
for layer in model.layers[:15]:
  layer.trainable = False # 15層までは学習しない
# ループ終わり
print('# laers=',len(model.layers)) # モデルの層の数を表示

# laers= 20


## 学習モデルをコンパイルする

In [ ]:
# 損失はクロスエントロピー法で算出、最低
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

## コンパイルしたモデルを表示

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## コンパイルしたモデルで収集した画像データを学習させる

In [ ]:
model.fit(X_train,Y_train, epochs=20, batch_size=16)

Epoch 1/20
8/8 [==============================] - 12s 245ms/step - loss: 0.9566 - accuracy: 0.4583
Epoch 2/20
8/8 [==============================] - 1s 90ms/step - loss: 0.6982 - accuracy: 0.5417
Epoch 3/20
8/8 [==============================] - 1s 90ms/step - loss: 0.6962 - accuracy: 0.4583
Epoch 4/20
8/8 [==============================] - 1s 95ms/step - loss: 0.6932 - accuracy: 0.4417
Epoch 5/20
8/8 [==============================] - 1s 101ms/step - loss: 0.6932 - accuracy: 0.5083
Epoch 6/20
8/8 [==============================] - 1s 100ms/step - loss: 0.6931 - accuracy: 0.5083
Epoch 7/20
8/8 [==============================] - 1s 97ms/step - loss: 0.6932 - accuracy: 0.5083
Epoch 8/20
8/8 [==============================] - 1s 91ms/step - loss: 0.7029 - accuracy: 0.4917
Epoch 9/20
8/8 [==============================] - 1s 91ms/step - loss: 0.6932 - accuracy: 0.5083
Epoch 10/20
8/8 [==============================] - 1s 92ms/step - loss: 0.6939 - accuracy: 0.5083
Epoch 11/20
8/8 [========

In [ ]:
score = model.evaluate(X_test, Y_test, batch_size=16)

2/2 [==============================] - 2s 2s/step - loss: 0.6933 - accuracy: 0.4839


### 作成した学習モデルとクラス分類名を保存

In [ ]:
import pickle
# クラス分類の保存
pickle.dump(classes, open('classes.sav','wb')) # オブジェクトをバイナリファイルで保存
# 学習モデルの保存
model.save('cnn,h5')

## 保存したクラス分類と学習モデルを読み込んで使う

In [24]:
# ライブラリのimport
from keras.models import load_model
import pickle,cv2,glob

## ファイルを読み込んでオブジェクトとしてよみがえらせる

In [25]:
model = load_model('cnn,h5')
classes = pickle.load(open('classes.sav', 'rb'))

## Sample画像を読み込む

In [26]:
files = glob.glob('sample/*')
print(files)

['sample/cat_82.jpg', 'sample/dog_89.png', 'sample/dog_87.png']


## ファイルパスから画像を読み込み、学習モデルに判定させ、該当するoutputフォルダにコピーを書き込む

In [28]:
for file in files:#ファイルの数だけループ
  img = cv2.imread(file) #画像データを読み込む
  #判定のためにimgを加工したimg2を作る
  img2 = cv2.resize(img,dsize=(224,224))#(224,224)にリサイズ
  img2 = img2.astype('float32')
  img2 = img2/255.0 #0.0～255.0だったデータを0.0～1.0に変換
  img2 = img2[None,...]#1次元配列に変換
  #正規化したデータを判定
  result = model.predict(img2)
  print(result)#画面に生で出力
  #確率が高いクラスを取得
  pred = result.argmax()
  print(pred,str(classes[pred]))
  #判定したクラス分類のフォルダのほうに、画像データを書き込む
  cv2.imwrite('output/'+str(classes[pred]) + '/' + file, img)

1/1 [==============================] - 1s 822ms/step
[[0.50283235 0.49716765]]
0 dog
1/1 [==============================] - 0s 17ms/step
[[0.50283235 0.49716765]]
0 dog
1/1 [==============================] - 0s 18ms/step
[[0.50283235 0.49716765]]
0 dog
